### Project 1: Data Exploration, Visualization and Dimensionality Reduction
## Animal Shelter Outcomes: A Classification Problem
#### Alex Matsunami, RJ Smith, Cory Nichols

## Business Understanding


Millions of animal companions end up in shelters each year in the United States. These animals may be stray, rescued from a difficult situation or given up by their previous families. Each year, roughly 40 percent of all dogs and cats in shelters are euthanized. 

Our dataset comes from the Austin Animal Center in Austin, Texas and is part of a Kaggle competition. Using this data, we hope to find trends or rule sets identifying the outcomes of animals in shelters. The findings would subsequently drive shelter practices, hopefully for more social good than business good. For instance, if spaying or neutering a dog or cat is highly suggestive of a favorable outcome for an animal, shelters should implement policies ensuring all animals are fixed prior to adoption. This practice may cost the shelter more money, but would improve the welfare of the animal population. Further, could something as simple as giving each an animal a name drive a more favorable outcome? This particular practice would cost nothing but time and in turn improve the welfare of the animal population in cities. Cost savings, however, could be gained from our analysis. For instance, it may be easy enough to automate intake and processes for certain breeds or genders of animals while focusing attention on breeds or genders of animals that the data indicate "need more help."

We intend to use the 27 thousand records in the training set as a reference for other major cities in the United States, where most animal shelter practices are similar from an intake and outcome perspective. Our dataset includes statistics on breed, color, sex, and age from the Austin Animal Center.

Success should be categorized into analytical and business goals. Analytical goals would be based on classification accuracy on a sizeable test data set given by a confusin matrix where overall classification accuracy is greater than 85% at minimum with at least 85% precision. Further, and possibly more importantly, we hope our analysis drives at least a 20% decrease in animal euthanizations over a two calendar year period.  


## Data Meaning and Data Types

### Overview

The original shelter outcomes dataset obtained from Kaggle.com contained the following ten attributes: Animal ID, Name, DateTime, OutcomeType, OutcomeSubtype, AnimalType, SexuponOutcome, AgeuponOutcome, Breed, and Color. The training dataset in use is 26,729 records while the test dataset contains roughly 12,000 records. We will be utilizing the training dataset. Each attribute in the training dataset is categorical, with AgeuponOutcome to be converted to continuous. 

### The Target Feature

OutcomeType is a nominal attribute and our target variable / feature with five levels: Return_to_owner, Euthanasia, Adoption, Transfer, and Died. Adoption is the level of OutcomeType our business clients are particularly interested in. 

OutcomeSubtype is a nominal attribute with 16 categories which represent subcategories of our target variable: OutcomeType. Data is missing for roughly half of the dataset for this feature. OutcomeSubtype is not a critical feature, contains too much missing data and too many levels to efficiently classify, therefore it will likely be dropped. 

### Explanatory Features

The explanatory features in our dataset are made up of AnimalID, Name, DateTime, AnimalType, SexuponOutcome, AgeuponOutcome, Breed and Color. Perusing the data at a high level, we can easily see that there are many opportunities to split the data into valuable, new features which will help train our classification algorithm more effectively.

Animal ID is a unique identifier given to every animal that enters a shelter. For the purpose of statistical analysis, the Animal ID's or substring of the animal ID does not provide valuable information and will be removed from the dataset. 

Name is a nominal attribute with 6374 different names represented in the dataset. This particular attribute may be valuable from a classification standpoint: does a pet being named increase the chances of adoption?

DateTime is a temporal attribute which denotes the time at which the outcome occurred. 

AnimalType is a nominal attribute with 2 categories: Cat and Dog and represents a critical analysis point and classification explanatory feature.  

SexuponOutcome is a nominal attribute with 5 categories: Neutered Male, Intact Male, Spayed Female, Intact Female, and Unknown. This attribute can easily be split into separate nominal features indcating if an animal is fixed and the gender of the animal. 

AgeuponOutcome is an ordinal attribute with 44 levels, or different ages that were reported for animals upon outcome. In order to normalize this feature, we will attempt to convert AgeuponOutcome into days. Examples of levels in this feature include: 1 year, 15 days, 7 months. Each level begins with an integer in every case.

Breed is an ordinal attribute with 1380 categories that represent different breeds of both dogs and cats. Breed holds generic levels such as "domestic shorthair" for dogs, which may indicate an inability to truly determine the breed type. We will need to be cognizant of this fact when training our classification model. This attribute holds many opportunities for simplification and feature creation and transformation. For instance, we can identify breeds with a 'reputation', size of the animal and popular breeds.

Finally, color is an ordinal attribute with 366 categories that represent the different combination colors of the animals entering the shelter. An example of a level of Color is "Brown/White," with the first color representing the primary color of the animal. Due to the number of levels in the Color feature, aggregation of the levels of color will be needed to effectively train our classification algorithm and derive business impact. 

## Data Preprocessing: Transformation and Statistics 

In [2]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
from sklearn.preprocessing import LabelEncoder,StandardScaler,Imputer
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
df = pd.read_csv('~/desktop/train.csv')
breedMap = pd.read_csv('~/desktop/breed_size.csv')

IOError: File C:\Users\rjsmith/desktop/train.csv does not exist

In [ ]:
df.info()


Our dataset is made up of 10 variables, many of which can be split or transformed into additional variables. Further, the majority of the variables are categorical: mostly nominal variables except for ordinal variables like Age and temporal variables like DateTime. Some variables, such as AnimalID, will provide no value in our analysis.

Our class variable is OutcomeType. We are interested in classifying animal outcome types given the other explanatory variables, which we will elaborate upon further.


In [ ]:
df.head(3)

### Data Quality and Cleanup

In [ ]:
print 'Data types and counts:\n\n', df.info(), '\n\n'
print 'Number of nulls in the data set: \n', df.isnull().sum(),'\n\n'
print '\Data Descriptions: Counts, Frequency'

df.describe()


Based upon the information retrieved from the DataFrame, we are dealing with roughly 27 thousand records in the training set. We are only interested in the "OutcomeType" as our y (classifier) data. Outcome subtype is a sparse drill down on the Outcome type with sub-codes which are not explained thoroughly. Therefore, we recommend dropping this value as it is aub-variable of the outcome variable we are interested with roughly 50% valid values.

The good news is: 40% of animals are being adopted, most of which are neutered male dogs about a year old. 

In regard to new feature creation, we observe multiple opportunities. For instance, it will be interesting to see if having a name affects outcome and if the type of breed is aggressive (has a "reputation"). We can also see converting the AgeuponOutcome being valuable. 

However, before we create or transform features, let's clean up the data a bit more. There are a few nulls in our dataset. Of concern are the nulls in AgeuponOutcome and SexuponOutcome, in which there are 18 and 1 null values, respectively. This is not a large concern given the size of our dataset and what looks to be good quality data overall. However, we will attempt to keep the age nulls in the dataset. Losing one object in the dataset due a null in SexuponOutcome is not a concern. Because we care about whether or not a pet is named, the nulls in Name are not a concern. Further, OutcomeSubtype is simply a click down of OutcomeType, which is our classifier, and is quite sparse. We are dropping OutcomeSubtype, we are not concerned with the nulls.


In [ ]:
del df['OutcomeSubtype'], df['AnimalID'] # drop extraneous data  columns
df = df.dropna(subset=['SexuponOutcome']) # drop the one nan row in the sex column. Not a large impact to analysis.
df = df.reset_index() # reset index to ensure further operations are executed correctly

Categorical Filling Strategy:
Fill the nulls with top 3 categories in a column vector randomly for age and sex.
Outside of the nulls, there is a string for age: "0 years" with 22 instances. This provides about "0" value. We will also fill "unknowns" in the sex column vector with the same logic. We will fill these columns by iterating over and randomly choosing one value of the top 3 values instead of assigning an arbitrary value. We believe this is relevant because the top 3 categories are spread relatively similarly, with roughly similar percentages of each column vector.

In [ ]:
from collections import Counter
from random import randint
# pandas groupby - group dataset by common attributes - what's the mode 
print 'Number of unique, non nan ages:', len(np.unique(df.AgeuponOutcome[~pd.isnull(df['AgeuponOutcome'])])) 

randChooser = []
for i in ['AgeuponOutcome', 'SexuponOutcome']:
    randChooser.append(Counter(df[i].values).most_common(3)) # get the 3 most common categories

# randomly pick one of the three categories and replace null
df['AgeuponOutcome'] = [randChooser[0][randint(0,2)][0] 
                        if pd.isnull(i) or i.lower() == '0 years' else i for i in df['AgeuponOutcome'].values]
 
df['SexuponOutcome'] = [randChooser[1][randint(0,2)][0]\
                        if i.lower() == 'unknown' else i for i in df['SexuponOutcome'].values]

In [ ]:
del df['index']
print 'Number of nulls in the data set: \n', df.isnull().sum(),'\n'
df.head()

## Data Creation and Transformation


All columns outside of the name column are now non-null.

Let's create some useful variables in our data set. It would be interesting to investigate whether or not a potential adoptee having a name has an effect on their outcome.

The gender and what we will call "is_intact" (whether an animal is intact (not fixed)) are also interesting variables to investigate.

Further, it would be better to represent age as a continuous ratio variable. We will transform age into day and check it's distribution for normality.


In [ ]:
print 'There are %d unique names in the data set' % len(np.unique(df['Name'].values))

# 6375 unique names. That is quite a few names. Let's generalize into a binary feature called has_name instead:
# per Eric's feedback, use pandas functionality instead and do this more efficiently vs pure python list comps:

df['has_name'] = df['Name'].apply(lambda x: 0 if pd.isnull(x) else 1)

# Further, let's split out the intactness of an animal: whether or not the animal has been spayed or neutered
# as well as the gender:

intactness = lambda x: 'Fixed' if x.split()[0] in ['Spayed','Neutered'] else 'Intact'
df['is_intact'] = df['SexuponOutcome'].apply(intactness)

df['gender'] = df['SexuponOutcome'].apply(lambda x: x.split()[1])

# we will also convert ageuponoutcome into days:
df['AgeValue'] = df['AgeuponOutcome'].apply(lambda x: x.rstrip('s').split())
            
ageMap = {'year': 365, 
          'week': 7, 
          'month': 30,
          'day': 1} 

df['AgeValue'] = df['AgeValue'].apply(lambda x: ageMap.get(x[1])*int(x[0]))


Let us also identify potentially 'aggressive' breeds using a bit of regex and top 10 aggressive breeds list: 
http://www.therichest.com/rich-list/the-biggest/the-worlds-10-most-dangerous-dog-breeds/?view=all


In [ ]:
import re

agg_breeds = ['Bull','Doberman','Rottweiler','Husky','German','Boxer','Malamute','Dane']

df['is_aggressive'] = [re.split('\W+',i) for i in df['Breed']]

for idx, i in enumerate(df['is_aggressive'].values):
    if len(agg_breeds) != len(set(agg_breeds).difference(i)):
        df.set_value(idx, 'is_aggressive', 1)
    else:
        df.set_value(idx, 'is_aggressive', 0)      

Let's also grab the primary color using same regex approach and reduce the number of categories in color

In [ ]:
colorMap = {'exotic':['Agouti','Calico','Lynx','Tricolor','Tortie','Flame','Torbie'], # these are some odd colors...
            'dark':['Black','Chocolate','Liver','Sable','Seal','Red'],
            'neutral':['Gray','Silver','Tan','Buff','Brown','Fawn'],
            'light':['Cream','Blue','Pink','White','Yellow','Lilac','Ruddy','Apricot','Orange','Gold']}
# use pandas built ins to munge the data a bit faster - use rename, split_interval to convert age
# 
df['primary_color'] = [re.split('\W+',i)[0] for i in df['Color']]

for idx, color in enumerate(df['primary_color'].values):
    for k,v in colorMap.items():
        if color in v:
            df.set_value(idx,'primary_color', k)

One other critical attribute we are missing is animal size. We assume the size of an animal is extremely important, especially when it comes to adopting dogs. Therefore, we have retrieved the American Kennel Club's (http://www.akc.org/dog-breeds/) listing of breeds and associated sizes. Because there are so few toy breeds listed, we combined toy and small into "small." For breeds that were not represented in AKC's list (there were about 20), each  was mapped to size buckets based on the following logic: Small (avg weight <= 20 lbs ), Medium (<= 65 lbs), Large (<= 100 lbs), Giant (> 100 lbs). All cats were mapped to "small." We expect the size feature to be more relevant for dogs, and we expect it to be significant. We've also added in an indicator for the top 10 most popular dogs.

In [ ]:
# set primary breed as the lookup
df['BreedLU'] = [i.rstrip('Mix').split('/')[0].strip() for i in df['Breed']] 

# zip up breed and size into a dict
sizeMapper = dict(zip(breedMap['Breed'], breedMap['Size'])) 

# map it, I love Pandas
df['size'] = df['BreedLU'].map(sizeMapper) 
df['size'] = np.where(df['AnimalType'].values == 'Cat', 'Small', df['size']) # set cats to small (unless Garfield)
del df['BreedLU'] # drop the lookup vector

In [ ]:
# Source: http://www.petinsurance.com/healthzone/pet-articles/pet-breeds/Top-10-Dog-Breeds.aspx
topBreeds = ['Boxer', 'Rottweiler', 'Poodle', 'Yorkshire Terrier',
                'French Bulldog', 'Beagle', 'Bulldog',
                'Golden Retriever', 'German Shepherd', 'Labrador Retriever']

topBreedList = []
for i in df['Breed']:
    matched = False
    for topBreed in topBreeds:
        if topBreed in i:
            topBreedList.append(1)
            matched = True
            break
    if not matched:
        topBreedList.append(0)       

df['is_popular'] = topBreedList

Now that we've transformed our dataset. Let's drop a few extraneous columns such as SexUponOutcome, since it is now split between is_intact and gender.

## Variable Investigation And Single Variable Data Visualization

First, let's convert categorical features to objects and make column names a little easier to follow:

In [ ]:
df.columns = ['Name','DateTime','Outcome','Type','SexuponOutcome','AgeuponOutcome','Breed','Color','Has_Name',
              'Is_Intact','Gender','Age_Value','Is_Aggressive','Primary_Color','Size', 'Is_Popular']

categ = ['Type','Breed','Is_Intact','Gender','Is_Aggressive', 'Primary_Color', 'Size','Has_Name',
         'Outcome','Is_Popular']

df[categ] = df[categ].astype(np.object)
df[categ].describe()

The data is starting to tell more of a story with our transformed dataset. There are no longer nulls and it looks as if mostly small dogs with names make up a good portion the data set. Nearly 70% of animals in our dataset are fixed. Interestingly, domestic shorthair mixes dominate the data set from a breed perspective, which have 1380 unique values. This may be a "catch all" breed used to identify dogs the shelter had trouble identifying and may be a future issue from a classification standpoint.

We also transformed the age variable into a continuous feature. Let's ensure it has a valid data type and quickly take a look at its distribution and some other key statistics.

In [ ]:
df['Age_Value'] = df['Age_Value'].astype(float)
df['logAge'] = np.log(df['Age_Value']).astype(float)
df['sqrtAge'] = np.sqrt(df['Age_Value']).astype(float)
df[['Age_Value','logAge','sqrtAge']].describe()

On average, animals are about two years old when adopted. Based simply on the range, age needs to be transformed if we are to use it as a continuous variable. Some animals are one day old upon outcome, while some are as old as 7300 days (20 years!). We will attempt two common methods: the natural log and square root transforms. We will use these transformations for ease of explanation to the business when back transforming and ensuring a more normal distribution for use in classification algorithms. 

Because age was technically a many-leveled categorical variable transformed to a continuous variable, we expect to see a stepped distribution. Because of this we will likely run algorithms with both the continuous and categorical versions of age.

First lets look at the original distribution of age, the log age and sqrt of age in violin plots:

In [ ]:
from random import random
c = ['firebrick','lightgreen','lightblue']
atts = ['Age_Value','logAge','sqrtAge']

for i in range(2):
    plt.figure(figsize=(20,15))
    for index, plots in enumerate(atts):
        plt.subplot(5, 4, index + 1) # plot them all on one row
        if i == 0:
            sns.violinplot(y = plots, data= df, color = c[index])
        else: sns.distplot(df[plots])
plt.show()

Violin plots allow us to account for multi-modality while also giving us standard box plot information. These charts plot kernel density estimates on each side of the box plot. As we can see above, our data is very much multi modal.  We also plot histograms and overlayed KDEs and end up with the same outcome. Let's also look at EDFs and normality tests to confirm a lack of age normality:

In [ ]:
plt.figure(figsize=(20,10))
for index, feature in enumerate(['Age_Value','logAge','sqrtAge']):
    plt.subplot(3, 3 , index + 1)
    x_ax = np.sort(df[feature]) # sort the feature ascending
    y_ax = np.cumsum(x_ax) # take cumulative sum, this creates an array of each observation adding to total sum
    y_ax = y_ax / np.max(y_ax) # get each observation's percentage of total sum of attribute
    plt.plot(x_ax,y_ax) # plot it
    plt.title(feature) # give it a title
    
plt.show()

# It's apparent age is not normally distributed and the transforms dont do much to help us.

In [ ]:
from scipy.stats import normaltest
vars = ['Age_Value','logAge','sqrtAge']
for i in vars:
    print i.ljust(10), 'chi-square:', round(normaltest(df[i])[0]), '\tp-value: {:.2f}'.format(normaltest(df[i])[1])

As shown above, the numerically converted Age is right skewed with a median value of about 365 days. This skewness is attributed to the younger ages of adopted animals. Empirical distribution functions show our data in a more quantitative fashion: showing probabilities of of obtaining values on the y axis and the value range itself on the x-axis. For instance, there is roughly an 85% chance of finding an age less than 4000 days in our dataset. We are looking for a relatively smooth upward curve that is not skewed severely 

Transforming the feature Age into a log representation and sqrt representation doesn't provide gaussian distributions, confirmed by EDF plots and D'Agostino's normal test. Depending on the classification algorithm used, we will likely need to utilize a categorical variable with levels for Age. 

In [ ]:
# let's convert age to an age range, if an animal is a year or less: young, up to 8 years: adult, 8+ years: senior
df['age_range'] = pd.cut(df.Age_Value/365,[0, 1, 8, 1e6],3, labels=['young', 'adult', 'senior'])
del df['logAge']
del df['sqrtAge']

Now that we've thoroughly analyzed and visualized age, we will move on to categorical variables. The rest of our dataset is primarily categorical (both nominal and ordinal variables). Visualizations of counts will be the priority. Of interest in particular are the variables we spent a significant amount of time creating, such as size and aggressive breeds.

To be thorough let's go ahead and plot percentage bars and investigate counts, which is common for individual categorical variables:

In [ ]:
import matplotlib

features = ['Has_Name','Is_Intact','Gender','Size','Is_Aggressive','Primary_Color', 'Is_Popular','Outcome','age_range']

plt.figure(figsize=(30,18))
for index, feature in enumerate(features):
    plt.subplot(3, 4, index +1)
    groups = df.groupby(feature)['Outcome'].count()/len(df)
    ax = groups.plot(kind = 'bar', color =['tomato','lightblue','lightgreen','gray','orchid'], fontsize=14)

In [ ]:
df.Type.value_counts().plot(kind='pie', autopct='%1d%%', colors = ['g', 'c'])

Some interesting discoveries come out of simple percentages: for instance, most animals being adopted are small. This is likely because cats are also categorized into the small category -- we will explore animal type crosstabs more thoroughly. Most critically, adoption rates show promise: 40 percent of all animals are adopted. Our business partners are looking to increase this number. Further, given Austin's policy of no euthanizations unless aggressive or sick, it's easy to see that most animals do indeed survive, however, most are transferred to other shelters. Transfers come with high cost for our business partners. This is likely a metric they'd like to reduce. The more an animal is passed through the system, the larger emotional detriment and the higher cost for the shelters in managing input/output as well as standard medical treatment, room and board and surgical operations. Further, it looks as though the most popular breeds are not present in the sample of shelter animals, which could be a bad sign for future adoptability.

Based on these charts, we also likely can combine size into a three level factor feature by combining giant into large since it is about 2% of the dataset. Further, most animals are dark colored and fixed. Fixed animals make up almost 75% of the dataset while dark colored animals are 40% of the sample size.

Further, there are more dogs than cats in our dataset. This is actually a surprise, given that stray cats are far more frequently encountered than stray dogs. 

## Feature Relationships, Correlative Analysis and Further Visualizations

Visualizing relationships of categorical variables can be difficult. However, we're going to start with a mosaic plot (tree map) to show relative sizes of categorical variables and how they interact with one another. First, we are going to investigate the type of animal, split and sized by Is_Intact and Size

In [ ]:
# mosaic plot for dogs on size and color
from statsmodels.graphics.mosaicplot import mosaic
for i in np.unique(df['Type']):
    mosaic(df[df['Type'] == i], ['Is_Intact','age_range']),
    plt.title(i+'s')
    plt.show()

We immediately see that most animals are fixed, which is something we could tell with univariate analysis. However, for both dogs and cats, we notice that younger animals in general have a higher rate of being intact. This is likely because the animals are newer arrivals, however, our dataset does not contain temporal data in order to determine this effect. 

In [ ]:
dfdogs = df[df.Type == 'Dog']
dogs = pd.crosstab([dfdogs['Size'], dfdogs['age_range']], dfdogs.Is_Aggressive.astype(bool))
dogs_rate = dogs.div(dogs.sum(1).astype(float), axis=0)*100
print dogs_rate
dogs_rate.plot(kind='barh', stacked=True, color =['lightblue','orange'], legend=False)
plt.title('Dogs: Size, Age and Aggressive Reptuation')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

Here, we are looking at dog age ranges and aggressiveness. Some interesting trends are readily apparent: large, adult dogs tend to be more 'aggressive' breeds, even moreso than giant dogs. As expected, small dogs don't have much of a reptuation of being 'dangerous' or aggressive. 

### Additional Work: Algorithmic Setup and Numeric Conversions

To make things a bit easier to visualize in seaborn and also set up our data for classification algorithms, let's go ahead and set up dummies and remap some of our categorical variables to integers where appropriate.

In [ ]:
# size is ordinal, lets create a dict and map it
sizeMap = {'Giant':4,'Large':3,'Medium':2,'Small':1}
df['Size'] = df['Size'].map(sizeMap)

# grab dummies for our nominal variables
df = df.join(pd.get_dummies(df['Primary_Color']))
df = df.join(pd.get_dummies(df['Type']))

# relabel our outcomes
classLabels = LabelEncoder()
df['Outcome_Label'] = classLabels.fit_transform(df['Outcome']) 
df['Primary_Color_Label'] = classLabels.fit_transform(df['Primary_Color']) 


Now, let's take a look at more relationships between our explanatory features. Specifically the age_value, size and intactness of dogs:

In [ ]:
sns.lmplot("Age_Value", "Size", data=df[df.Type=='Dog'], hue = 'Is_Intact', x_jitter=.2, y_jitter =.2, fit_reg=False)

This interesting scatterplot gives us a number of different pieces of valuable information on the dogs in our dataset. First, Smaller dogs make are the most frequently occurring of the sizes in our dataset, and also contain the oldest of the dogs. By jittering our data points, it looks as if larger sized dogs are fixed more often than other sizes. We also can easily confirm that younger dogs are intact more often than older dogs.

In [ ]:
#df[df.Type=='Cat']
print 'Lookup:' 
print np.unique(classLabels.inverse_transform(df['Primary_Color_Label']))
g = sns.FacetGrid(df[df.Type =='Cat'], col = "age_range", row= "Gender", margin_titles=True)
g.map(plt.hist, "Primary_Color_Label" , color= "lightblue")

Looking at the distributions of our categorical variables, age range, gender and color, we can see that light colored, young males make up a large portion of our cats. There are few senior aged cats in our dataset in comparison to young cats. This is likely due to the high birth rates and litter sizes of cats. It is often common to see many kittens and few old cats in animal shelters when adopting. Interestingly, there are also many exotic colored female cats.

## Feature and Target: Visual Relationships and Analysis

Let's take a look at a formal test and visualization to see how our features relate to the outcome label. By using a chi-square test of independence, we will attempt to determine if our features are truly independent (not related) to outcome or if there is a relationship (dependency). Significance thresholds are set at .05.

In [ ]:
np.minimum(len(df)-1,len(cols)-1)

In [ ]:
import scipy

cols = ['Type','Has_Name','Is_Intact','Gender','Is_Aggressive','Primary_Color','Is_Popular','age_range','Size']

def chisqr(df, features):
    vals = []
    for i in features:
        cross = pd.crosstab(df[i], df['Outcome_Label'])
        chisq = scipy.stats.chi2_contingency(cross)
        cramer = np.sqrt((chisq[0]/len(df)) / np.minimum(len(df)-1,len(cols)-1))
        chisq = list(chisq[0:2])
        chisq.append(cramer)
        vals.append(chisq)
    return DataFrame(vals)

chi2 = u'\N{GREEK SMALL LETTER CHI}\N{SUPERSCRIPT TWO}'
plt.figure(figsize=(10,10))
hm = sns.heatmap(chisqr(df,cols),cbar=True,annot=True,square=True, fmt='.2f',annot_kws={'size':15},
                 yticklabels=cols,xticklabels=[chi2 + ' Score','p-value','Cramer\'s V'])
plt.show()

Based on a chi-squared test of independence, we determine that our categorical features are all significantly related to our target variable Outcome. In fact, the most dependent variable is Is_Intact, essentially stating that Outcome is highly dependent upon whether or not an animal is fixed. We also take a look at the effect size to determine the strength of association. As we can see, the strength of association between Is_Intact and Outcome is the highest at .19, which is only moderately associated. Most of the other categorical variables are either weakly or moderately associated with outcome. Of the variables, Is_Intact, age_range, has_name and Type would be our best predictors. Surprisingly, gender's effect size is quite small -- insignificant in fact, as well as the primary color of the animal.

Let's cut our visualizations a couple different ways now, based on these results.

In [ ]:
animals = pd.crosstab(df['Is_Intact'], df['Outcome'])
rate = animals.div(animals.sum(1).astype(float), axis=0)*100
rate.plot(kind='barh', stacked=True)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

plt.show()

In [ ]:
animals = pd.crosstab(df['Outcome'], df['Is_Intact'])
rate = animals.div(animals.sum(1).astype(float), axis=0)*100
rate.plot(kind='barh', stacked=True)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

plt.show()

Probably the most significant finding thus far: it looks as if over 50 percent of fixed animals are adopted. Specifically, looking at the second chart, over 90 percent of adopted animals are fixed. Given the moderate association found for Is_Intact, this is a critical datapoint discovered for our classification purposes going forward. Further, the majority of animals that perished were intact. This could be due to pre-existing conditions coming into the facility.

In [ ]:
animals = pd.crosstab(df['Type'], df['Outcome'])
rate = animals.div(animals.sum(1).astype(float), axis=0)*100
rate.plot(kind='barh',stacked=True)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

Cats are transferred far more often than dogs, however, both are adopted at similar rate. Dogs are much more likely to be returned to their owners than cats. Of interest is the deaths among cats and dogs: only cats have died in our dataset. Thankfully both cats and dogs do not die or are not euthanized often. 

In [ ]:
animals = pd.crosstab(df['age_range'], df['Outcome'])
rate = animals.div(animals.sum(1).astype(float), axis=0)*100
rate.plot(kind='barh',stacked=True)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

As one could probably expect, young animals are adopted at higher rates than adult or senior animals. The highest rates of euthanasia occur in senior animals, likely due to failing health. Return to owners are also high in senior animals, most likely due to the inability for more senior animals to be adopted. Because age_range has a slight association with outcome and is one more the more associated features in our dataset, these are valuable findings prior to executing classification procedures.

In [ ]:
X = dfdogs['age_range']